In [2]:
import numpy as np
import pandas as pd
import requests  
import json
import glob

import os

In [14]:
#####################predict

In [ ]:
import pandas as pd
import requests, os
from pathlib import Path

# ----------------- 可选：把你的函数做成一个独立模块 -----------------
def process_one_file(csv_path: Path, out_dir: Path):
    """
    读取 benchmark_weights/{theme}.csv
    运行整段计算
    保存结果到 weighted_return/{theme}_benchmark_weights.csv
    """
    theme = csv_path.stem        # 例如 "Gold"
    print(f"=== Deal with {theme} ===")

    # ---------- 1) 读入权重 ----------
    gold = pd.read_csv(csv_path)
    #gold=pd.read_csv('benchmark_weights\Gold_benchmark_weights.csv')
    weight_data=gold.copy()#pd.read_csv(f"benchmark_weights\{theme}_benchmark_weights.csv")

    r = requests.post('http://flowrider.prod.schonfeld.com:8000/api/data/mkt-data-history', json={
            'bbg_list': gold['bbg'].drop_duplicates().tolist(),
            'start_date': '2023-05-15',
            'end_date': '2025-05-15'})
    rt_1 = pd.DataFrame(r.json())

    #missing stocks
    gold_set = set(gold['bbg'])
    rt_1_set = set(rt_1['bbg'])
    #在gold中但不在price_1_set中
    missing_stocks =  gold_set - rt_1_set 
    if not missing_stocks:                        # empty set / list / Series
        rt_all_d = rt_1.copy()                    # nothing to fetch, keep original
        #find missing stock from another API
    else:
        miss_stock_list = list(missing_stocks)#.tolist()

        # Add " Equity" to each item
        miss_stock_list_equity = [item + " Equity" for item in miss_stock_list]

        # Mandatory fields
        securities = miss_stock_list_equity #['000046 CH Equity']
        fields = ['CHG_PCT_1D']
        start_date = 20230515
        end_date = 20250515

        # Optional fields
        # overrides = {}
        overrides = {
            'EQY_FUND_CRNCY': 'USD',

        }
        # options = {}
        options = {
            'nonTradingDayFillOption': 'ALL_CALENDAR_DAYS',
            'nonTradingDayFillMethod': 'PREVIOUS_VALUE'
        }

        # Make the API call using Python's requests package
        url = 'http://flowrider.prod.schonfeld.com:8005/bdh'
        payload = {
            'securities': securities,
            'fields': fields,
            'start_date': start_date,
            'end_date': end_date,
            'overrides': overrides,
            'options': options
        }
        response = requests.post(url, json=payload)
        rt_miss_d=pd.DataFrame(response.json()['content'])

        # Make a copy of rt_miss to avoid modifying the original
        rt_miss_transformed = rt_miss_d.copy()
        rt_miss_transformed['date'] = pd.to_datetime(rt_miss_transformed['date']).dt.strftime('%Y-%m-%d')
        rt_miss_transformed['security'] = rt_miss_transformed['security'].str.replace(' Equity', '')
        rt_miss_transformed = rt_miss_transformed.rename(columns={'security': 'bbg'})
        rt_miss_transformed = rt_miss_transformed.rename(columns={'CHG_PCT_1D': 'px_chg_1d'})

        # Divide the px_chg_1d values by 10
        rt_miss_transformed['px_chg_1d'] = pd.to_numeric(rt_miss_transformed['px_chg_1d'], errors='coerce')
        rt_miss_transformed['px_chg_1d'] = rt_miss_transformed['px_chg_1d'] / 100

        #concat with rt_1
        rt_all_d=pd.concat([rt_1, rt_miss_transformed], ignore_index=True)
        rt_all_d=rt_all_d.sort_values(by=['date','bbg']).reset_index(drop=True)

    df_d=rt_all_d
    theme_data = df_d


    # # 创建透视表，行为日期，列为股票代码，值为价格变化
    pivot_data = theme_data.pivot_table(
        index='date',
        columns='bbg',
        values='px_chg_1d'#'px_chg_1w'
    )


    # Calculate the two types of weighted returns
    #equal_weighted_returns = pivot_data.apply(calculate_equal_weighted_return, axis=1)
    def calculate_freefloat_weighted_return(date_row):
        total_weighted_return = 0
        
        for bbg in date_row.index:
            if pd.notna(date_row[bbg]):  # Check if return is not NaN
                if bbg in weight_data['bbg'].values:
                    weight = weight_data.loc[weight_data['bbg'] == bbg, 'benchmark_2_weight'].values[0]
                    total_weighted_return += date_row[bbg] * weight
        
        return total_weighted_return

    free_weighted_returns = pivot_data.apply(calculate_freefloat_weighted_return, axis=1)

    # Create a new DataFrame with both columns
    combined_returns_d = pd.DataFrame({
        #'weighted_return_1': equal_weighted_returns,
        'weighted_return_2': free_weighted_returns
    })
    # ---------- 5) 保存 ----------
    out_dir.mkdir(exist_ok=True)
    out_path = out_dir / f"{theme}_return.csv"
    combined_returns_d.to_csv(out_path, index=True)
    print(f"已保存: {out_path}")

# ----------------- 主脚本 -----------------
def main():
    in_dir  = Path("benchmark_weights")
    out_dir = Path("weighted_return")

    for csv_file in in_dir.glob("*.csv"):
        process_one_file(csv_file, out_dir)

if __name__ == "__main__":
    main()

In [ ]:
volume_df=pd.read_csv('market_data_history_202505201802.csv')
volume_df= volume_df.sort_values(["date", "bbg"])
volume_df = volume_df[
    (volume_df["date"] >= "2023-05-15") & 
    (volume_df["date"] <= "2025-05-15")
]
volume_df["date"] = pd.to_datetime(volume_df["date"])
volume_df = volume_df.sort_values(["bbg", "date"])

volume_df["vol_chg_pct"] = (
    volume_df
        .groupby("bbg")["volume"]
        .pct_change()         # (vol_t / vol_{t-1} - 1)
)
from pathlib import Path

DIR_W = Path("benchmark_weights")          # 目录路径，可改绝对路径

suffix  = "_benchmark_weights.csv"         # 需要剔除的后缀

bench_list = [

    f.stem.replace(suffix[:-4], "")        # stem 去掉".csv"，再删「_benchmark_weights」

    for f in DIR_W.glob("*_benchmark_weights.csv")

]



# ========== 可选：要循环的 Benchmark 名称 ==========
benchmarks = bench_list             # 需要几个就写几个，例如 ["Gold", "Silver", "Oil"]
# ========== 可选：存放文件夹 ==========
DIR_W  = Path("benchmark_weights")
DIR_R  = Path("weighted_return")
# ----------------------------------------------------
def weighted_row_sum(row_vals, row_wts):
    """对一行做加权平均；只对非 NaN 的列参与计算"""
    mask = row_vals.notna()
    if mask.sum() == 0:
        return pd.NA
    adj_w = row_wts[mask]
    adj_w = adj_w / adj_w.sum()            # 权重归一
    return (row_vals[mask] * adj_w).sum()
def run_one_benchmark(bench_name: str) -> pd.DataFrame | None:
    """返回单个 benchmark 的日度特征表 final_df（列：date, ret, volume, turnover,
       crowd, crowd_c, mom_1m, theme）；若缺文件则返回 None"""
    w_path = DIR_W / f"{bench_name}_benchmark_weights.csv"
    r_path = DIR_R / f"{bench_name}_benchmark_weights_return.csv"
    if not (w_path.exists() and r_path.exists()):
        print(f":warning:  {bench_name}: 找不到权重或收益文件，跳过")
        return None
    # ---------- 基础数据 ----------
    weight_df   = pd.read_csv(w_path)               # bbg, benchmark_2_weight
    gold_return = pd.read_csv(r_path)               # date, bbg, weighted_return_2
    weight_df["bbg"]      = weight_df["bbg"].astype(str)
    gold_return["date"]   = pd.to_datetime(gold_return["date"])
    idx_base = gold_return["date"].sort_values().unique()  # 统一基准索引

    # ========== ①  volume / turnover ==========
    sub = (weight_df[["bbg"]]
           .merge(volume_df, on="bbg", how="left")
           .sort_values(["date", "bbg"]))
    # 行有效性判断：date 有值且 volume/turnover 至少一个非空
    valid = sub["date"].notna() & sub[["vol_chg_pct"]].notna().any(axis=1)
    sub = sub.loc[valid]
    if sub.empty:
        daily_feat_vol = pd.DataFrame(np.nan,
                                      index=idx_base,
                                      columns=["vol_chg_pct"])
    else:
        vol_chg_pct_pivot = sub.pivot_table(index="date", columns="bbg",
                                    values="vol_chg_pct",   dropna=False)
        # tur_pivot = sub.pivot_table(index="date", columns="bbg",
        #                             values="turnover", dropna=False)
        # 统一重建权重矩阵
        w_vec = (weight_df.set_index("bbg")["benchmark_2_weight"]
                 .reindex(vol_chg_pct_pivot.columns))
        weight_mat = pd.DataFrame(np.tile(w_vec.values, (len(vol_chg_pct_pivot), 1)),
                                  index=vol_chg_pct_pivot.index,
                                  columns=vol_chg_pct_pivot.columns)
        idx_volume = vol_chg_pct_pivot.apply(
            lambda r: weighted_row_sum(r, weight_mat.loc[r.name]), axis=1)
        # idx_turnov = tur_pivot.apply(
        #     lambda r: weighted_row_sum(r, weight_mat.loc[r.name]), axis=1)
        daily_feat_vol = pd.DataFrame(
            {"vol_chg_pct": idx_volume},
            index=vol_chg_pct_pivot.index
        ).reindex(idx_base)              # 填补缺失日期

    # ========== ③  收益表 ==========
    gold_daily_ret = (gold_return
                    .pivot_table(index="date",
                                values="weighted_return_2",
                                aggfunc="first")
                    .reindex(idx_base))
    # ========== ④  拼 final_df ==========
    final_df = (gold_daily_ret
                .join(daily_feat_vol, how="left")
                .reset_index()
                .rename(columns={"index": "date",
                                "weighted_return_2": "ret"}))
    final_df["theme"] = bench_name
    return final_df

# ----------------------------------------------------
parts = []
for bench in bench_list:#[:3]:
    print(f"{bench} start")
    df = run_one_benchmark(bench)
    if df is not None:
        parts.append(df)
    #print(f"{bench} finish")
if not parts:
    raise RuntimeError("没有拼接出任何结果！")
result_df = pd.concat(parts, ignore_index=True)
print(f":heavy_check_mark: 全部完成，result_df 行数: {len(result_df):,}")
print(result_df.head())
result_df.to_csv('return_vol_chg_pct.csv')

In [ ]:
from pathlib import Path

DIR_W = Path("benchmark_weights")          # 目录路径，可改绝对路径

suffix  = "_benchmark_weights.csv"         # 需要剔除的后缀

bench_list = [

    f.stem.replace(suffix[:-4], "")        # stem 去掉".csv"，再删「_benchmark_weights」

    for f in DIR_W.glob("*_benchmark_weights.csv")

]



import pandas as pd

from pathlib import Path

import re

from datetime import datetime

# ------------ 用户可改的参数 ------------

ROOT = Path(r"X:\TeamRV-DON-US-RISK\Intern\Ruoxue\data\UBS\CROWDING_SCORES")

START = datetime(2023, 5, 15)

END   = datetime(2025, 5, 15)

KEY   = "ubs_crowding_scores_bloomberg_ticker"

USE_COLS = ['bloomberg_ticker', 'dt',

            'crowding_factor', 'crowding_factor_change', 'mom_1m']

# ----------------------------------------

def file_date(path: Path) -> datetime | None:

    """

    从文件名里提取形如 *_YYYYMMDD* 或 *_YYYYMM* 的日期，

    返回 datetime；提取失败则返回 None

    """

    m = re.search(r'_(\d{6,8})$', path.stem)

    if not m:

        return None

    ds = m.group(1)

    if len(ds) == 8:

        return datetime.strptime(ds, "%Y%m%d")

    if len(ds) == 6:

        return datetime.strptime(ds, "%Y%m")

    return None

def read_one(path: Path) -> pd.DataFrame:

    """

    读取单文件，只保留 USE_COLS，自动识别 csv / excel，

    并把列统一为小写

    """

    if path.suffix.lower() in (".xlsx", ".xlsm", ".xls", ".xlsb"):

        df = pd.read_excel(path)

    elif path.suffix.lower() == ".csv":

        df = pd.read_csv(path)

    else:

        raise ValueError(f"不支持的文件类型: {path}")

    df.columns = df.columns.str.lower()

    missing = [c for c in USE_COLS if c not in df.columns]

    if missing:

        raise KeyError(f"{path.name} 缺少列: {missing}")

    out = df[USE_COLS].copy()

    # 若 dt 不是日期型，尝试解析

    if not pd.api.types.is_datetime64_any_dtype(out['dt']):

        out['dt'] = pd.to_datetime(out['dt'], errors='coerce')

    out['src_file'] = path.name           # 方便溯源

    return out

# ------------ 主流程 ------------

all_parts = []

for p in ROOT.rglob("*"):

    if not p.is_file():

        continue

    name = p.name.lower()

    if KEY not in name:

        continue

    d = file_date(p)

    if d is None or not (START <= d <= END):

        continue

    try:

        part = read_one(p)

        all_parts.append(part)

        print(f"✔ 读取 {p.name}   行数={len(part)}")

    except Exception as e:

        print(f"⚠️  跳过 {p.name}  -- {e}")

if not all_parts:

    raise RuntimeError("未找到满足条件的文件！")

crowd_concat = pd.concat(all_parts, ignore_index=True)

print("\n=== 合并完成 ===")

print(f"总行数: {len(crowd_concat):,}")

print(crowd_concat.head())

crowd_concat['bloomberg_ticker']=crowd_concat['bloomberg_ticker'].str.replace(' Equity','')
crowd_concat=crowd_concat.drop(columns='src_file')
crowd_concat['date'] = pd.to_datetime(crowd_concat['dt'])
crowd_concat.sort_values('date', inplace=True)
crowd_concat=crowd_concat[['date','bloomberg_ticker',  'crowding_factor', 'crowding_factor_change', 'mom_1m']]
crowd_concat=crowd_concat.rename(columns={'bloomberg_ticker':'bbg'})
crowd_concat_df=crowd_concat.sort_values(["date", "bbg"])
crowd_concat_df = crowd_concat_df[
    (crowd_concat_df["date"] >= "2023-05-15") & 
    (crowd_concat_df["date"] <= "2025-05-15")
]



# ========== 可选：要循环的 Benchmark 名称 ==========
benchmarks = bench_list             # 需要几个就写几个，例如 ["Gold", "Silver", "Oil"]
# ========== 可选：存放文件夹 ==========
DIR_W  = Path("benchmark_weights")
DIR_R  = Path("weighted_return")
# ----------------------------------------------------
def weighted_row_sum(row_vals, row_wts):
    """对一行做加权平均；只对非 NaN 的列参与计算"""
    mask = row_vals.notna()
    if mask.sum() == 0:
        return pd.NA
    adj_w = row_wts[mask]
    adj_w = adj_w / adj_w.sum()            # 权重归一
    return (row_vals[mask] * adj_w).sum()
def run_one_benchmark(bench_name: str) -> pd.DataFrame | None:
    """返回单个 benchmark 的日度特征表 final_df（列：date, ret, volume, turnover,
       crowd, crowd_c, mom_1m, theme）；若缺文件则返回 None"""
    w_path = DIR_W / f"{bench_name}_benchmark_weights.csv"
    r_path = DIR_R / f"{bench_name}_benchmark_weights_return.csv"
    if not (w_path.exists() and r_path.exists()):
        print(f":warning:  {bench_name}: 找不到权重或收益文件，跳过")
        return None
    # ---------- 基础数据 ----------
    weight_df   = pd.read_csv(w_path)               # bbg, benchmark_2_weight
    gold_return = pd.read_csv(r_path)               # date, bbg, weighted_return_2
    weight_df["bbg"]      = weight_df["bbg"].astype(str)
    gold_return["date"]   = pd.to_datetime(gold_return["date"])
    idx_base = gold_return["date"].sort_values().unique()  # 统一基准索引

    # ========== ①  volume / turnover ==========
    sub = (weight_df[["bbg"]]
           .merge(volume_df, on="bbg", how="left")
           .sort_values(["date", "bbg"]))
    # 行有效性判断：date 有值且 volume/turnover 至少一个非空
    valid = sub["date"].notna() & sub[["volume", "turnover"]].notna().any(axis=1)
    sub = sub.loc[valid]
    if sub.empty:
        daily_feat_vol = pd.DataFrame(np.nan,
                                      index=idx_base,
                                      columns=["volume", "turnover"])
    else:
        vol_pivot = sub.pivot_table(index="date", columns="bbg",
                                    values="volume",   dropna=False)
        tur_pivot = sub.pivot_table(index="date", columns="bbg",
                                    values="turnover", dropna=False)
        # 统一重建权重矩阵
        w_vec = (weight_df.set_index("bbg")["benchmark_2_weight"]
                 .reindex(vol_pivot.columns))
        weight_mat = pd.DataFrame(np.tile(w_vec.values, (len(vol_pivot), 1)),
                                  index=vol_pivot.index,
                                  columns=vol_pivot.columns)
        idx_volume = vol_pivot.apply(
            lambda r: weighted_row_sum(r, weight_mat.loc[r.name]), axis=1)
        idx_turnov = tur_pivot.apply(
            lambda r: weighted_row_sum(r, weight_mat.loc[r.name]), axis=1)
        daily_feat_vol = pd.DataFrame(
            {"volume": idx_volume, "turnover": idx_turnov},
            index=vol_pivot.index
        ).reindex(idx_base)              # 填补缺失日期
        
    # ========== ②  crowding ==========
    sub = (weight_df[["bbg"]]
           .merge(crowd_concat_df, on="bbg", how="left")
           .sort_values(["date", "bbg"]))
    valid = (sub["date"].notna() &
             sub[["crowding_factor", "crowding_factor_change",
                  "mom_1m"]].notna().any(axis=1))
    sub = sub.loc[valid]
    if sub.empty:
        daily_feat_crowd = pd.DataFrame(np.nan,
                                        index=idx_base,
                                        columns=["crowd", "crowd_c", "mom_1m"])
    else:

        cf_pivot  = sub.pivot_table(index="date", columns="bbg",
                                            values="crowding_factor", dropna=False)
        cfc_pivot = sub.pivot_table(index="date", columns="bbg",
                                    values="crowding_factor_change", dropna=False)
        mom_pivot = sub.pivot_table(index="date", columns="bbg",
                                    values="mom_1m", dropna=False)
        w_vec = (weight_df.set_index("bbg")["benchmark_2_weight"]
                .reindex(cf_pivot.columns))
        weight_mat = pd.DataFrame(np.tile(w_vec.values, (len(cf_pivot), 1)),
                                index=cf_pivot.index, columns=cf_pivot.columns)
        idx_cf  = cf_pivot.apply(
            lambda r: weighted_row_sum(r, weight_mat.loc[r.name]), axis=1)
        idx_cfc = cfc_pivot.apply(
            lambda r: weighted_row_sum(r, weight_mat.loc[r.name]), axis=1)
        idx_mom = mom_pivot.apply(
            lambda r: weighted_row_sum(r, weight_mat.loc[r.name]), axis=1)
        daily_feat_crowd = pd.DataFrame(
            {"crowd": idx_cf, "crowd_c": idx_cfc, "mom_1m": idx_mom},
            index=cf_pivot.index
        ).reindex(idx_base)
    # ========== ③  收益表 ==========
    gold_daily_ret = (gold_return
                    .pivot_table(index="date",
                                values="weighted_return_2",
                                aggfunc="first")
                    .reindex(idx_base))
    # ========== ④  拼 final_df ==========
    final_df = (gold_daily_ret
                .join(daily_feat_vol, how="left")
                .join(daily_feat_crowd, how="left")
                .reset_index()
                .rename(columns={"index": "date",
                                "weighted_return_2": "ret"}))
    final_df["theme"] = bench_name
    return final_df

# ----------------------------------------------------
parts = []
for bench in bench_list:
    print(f"{bench} start")
    df = run_one_benchmark(bench)
    if df is not None:
        parts.append(df)
    #print(f"{bench} finish")
if not parts:
    raise RuntimeError("没有拼接出任何结果！")
result_df = pd.concat(parts, ignore_index=True)
print(f":heavy_check_mark: 全部完成，result_df 行数: {len(result_df):,}")
print(result_df.head())
result_df.to_csv('return_vol_crowd.csv')

In [ ]:
result_df=pd.read_csv('return_vol_crowd.csv')
result_df=result_df.set_index(result_df.columns[0])
 # 把第一列设为索引
result_df.index.name = None                            # 去掉索引名称
#result_df = result_df.drop(columns=result_df.columns[0])
result_df['date']=pd.to_datetime(result_df['date'])
result_df1 = result_df[~((result_df["date"].dt.weekday >= 5) & (result_df["ret"] == 0))].reset_index(drop=True)